# Course Recommender with SVD based similarity
> Applying SVD on education course dataset, storing in sqlite and wrapping in Flask REST api.

## Setup

In [ ]:
import os
import yaml
import copy
import json
import sqlite3
import logging
import requests
import functools
import numpy as np
import pandas as pd
from time import time
from typing import List
from flask import request
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import pairwise_distances

logging.getLogger().setLevel(logging.INFO)

In [ ]:
data_path = "."

# SQlite
env = "dev"
database = "recommender_dev.db"
username = "admin"
pwd = ""
hostname = "0.0.0.0"
port = 8081

similarity_metric = "cosine" # Similarity metric for pairwise distance measurement
weights = ['0.50','0.30','0.20'] # Weights for similarity matrix: interest,assessment,tags
results_table = "rank_matrix"  # SQLite3 table containing user similarity metrics
user_id = None # unique user id for scoring similarities

# Flask server
hostname = "0.0.0.0" # hostname for serving Flask application
port = 5000 # port for serving Flask application

## Data ingestion

In [ ]:
#hide-output
!wget https://github.com/sparsh-ai/user-recommender/raw/main/data/course_tags.csv
!wget https://github.com/sparsh-ai/user-recommender/raw/main/data/user_interests.csv
!wget https://github.com/sparsh-ai/user-recommender/raw/main/data/user_course_views.csv
!wget https://github.com/sparsh-ai/user-recommender/raw/main/data/user_assessment_scores.csv

In [ ]:
def ingest_raw_data(env: str, data_dir: str = "data"):
    """Write .csv raw files to SQLite Database"""
    csv_files = [i for i in os.listdir(data_dir) if ".csv" in i]
    for f in csv_files:
        df = pd.read_csv(os.path.join(data_dir, f))
        conn = sqlite3.connect(database)
        cur = conn.cursor()
        df.to_sql(name=f.split(".")[0], con=conn, if_exists="replace", index=False)

In [ ]:
ingest_raw_data(env, data_path)

## Load data from SQlite

In [ ]:
def read_table(env: str, query: str) -> pd.DataFrame:
    """Query Table from SQLite Database"""
    conn = sqlite3.connect(database)
    cur = conn.cursor()
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=[column[0] for column in cur.description])
    return df

In [ ]:
def load_data(env: str) -> dict:
    """Load Users and Content Data from SQLite"""

    df_course = read_table(env, f"select * from user_course_views")
    df_asmt = read_table(env, f"select * from user_assessment_scores")
    df_interest = read_table(env, f"select * from user_interests")
    df_tags = read_table(env, f"select * from course_tags")

    return {
        "course": df_course,
        "assessment": df_asmt,
        "interest": df_interest,
        "tags": df_tags,
    }

In [ ]:
# Load Users/Assessments/Course/Tags Data
data_raw = load_data(env)

## Summarize

In [ ]:
def data_summary(data: dict):
    """Print Summary Metrics of Data"""
    for name, df in data.items():
        logging.info(f"\nDataframe: {name.upper()} -- Shape: {df.shape}")
        for c in df.columns:
            unique = len(df[c].unique())
            is_null = df[df[c].isnull()].shape[0]
            logging.info(f"{c} -- Unique: {unique} -- Null: {is_null}")
    return

In [ ]:
# Summary of Users/Assessments/Courses/Tags Data
data_summary(data_raw)

INFO:root:
Dataframe: COURSE -- Shape: (249238, 6)
INFO:root:user_handle -- Unique: 8760 -- Null: 0
INFO:root:view_date -- Unique: 212 -- Null: 0
INFO:root:course_id -- Unique: 5942 -- Null: 0
INFO:root:author_handle -- Unique: 1412 -- Null: 0
INFO:root:level -- Unique: 3 -- Null: 0
INFO:root:view_time_seconds -- Unique: 11631 -- Null: 0
INFO:root:
Dataframe: ASSESSMENT -- Shape: (6571, 4)
INFO:root:user_handle -- Unique: 3114 -- Null: 0
INFO:root:assessment_tag -- Unique: 54 -- Null: 0
INFO:root:user_assessment_date -- Unique: 6570 -- Null: 0
INFO:root:user_assessment_score -- Unique: 282 -- Null: 0
INFO:root:
Dataframe: INTEREST -- Shape: (297526, 3)
INFO:root:user_handle -- Unique: 10000 -- Null: 0
INFO:root:interest_tag -- Unique: 748 -- Null: 0
INFO:root:date_followed -- Unique: 12869 -- Null: 0
INFO:root:
Dataframe: TAGS -- Shape: (11337, 2)
INFO:root:course_id -- Unique: 5942 -- Null: 0
INFO:root:course_tags -- Unique: 999 -- Null: 112


## Preprocessing

In [ ]:
def preprocess(data: dict) -> dict:
    """Preprocess input DataFrames"""
    prep = {}
    data = copy.deepcopy(data)
    for name, df in data.items():
        # drop null values
        df.dropna(axis=1, how="all", inplace=True)  # course tags table
        df.reset_index(drop=True, inplace=True)

        # rename columns in dataframe
        rename = {
            "interest_tag": "tag",
            "assessment_tag": "tag",
            "course_tags": "tag",
            "user_assessment_score": "score",
            "view_time_seconds": "view",
        }
        df.columns = [rename[i] if i in rename.keys() else i for i in df.columns]

        # discretize user assessment scores quantile buckets
        # if "score" in df.columns:
        #     df = df.replace({"score": {0:"low", 1:"medium", 2:"high"}})
        if any("score" in col for col in df.columns):
            df["score"] = pd.qcut(df["score"], q=3, labels=["high", "medium", "low"])

        # discretize user viewing time into quantile buckets
        # if "view" in df.columns:
        #     df = df.replace({"view": {0:"low",1:"high"}})
        if any("view" in col for col in df.columns):
            df["view"] = pd.qcut(df["view"], q=4, labels=["high", "medium", "low", "very low"])

        # encode categorical columns
        cat_cols = ["tag", "score", "view", "level"]
        for col in df.columns:
            if col in cat_cols:
                df[col] = pd.Categorical(df[col]).codes

        # save prep dataframe
        prep[name] = df

    # add key for max users -> used for initializing user-item matrix
    prep["max_users"] = max(
        [max(v["user_handle"]) for k, v in prep.items() if "user_handle" in v.columns]
    )

    # add key containing dataframe for merged course/tags
    prep["course_tags"] = pd.merge(
        prep["course"], prep["tags"], on="course_id", how="left"
    )
    return prep

In [ ]:
# Preprocess Raw Data
data = preprocess(data_raw)

## Calculating similarities and ranking

### Similarity

In [ ]:
class UserSimilarityMatrix:
    """Class for building and computing similar users"""

    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __repr__(self) -> str:
        return f"Dimensions of User-Items Matrix: {self.matrix.shape}"

    def build_user_item_matrix(self, max_users: str, item: str) -> None:
        """Build User/Item Interaction Matrix"""
        matrix = np.zeros(shape=(max_users, max(self.data[item])))
        for _, row in self.data.iterrows():
            matrix[row["user_handle"] - 1, row[item] - 1] = 1
        return matrix

    def get_user_item_matrix(self, max_users: int, features: List[str]):
        """Concatenate Features into One User-Items Matrix"""
        results = []
        for item in features:
            results.append(self.build_user_item_matrix(max_users, item))
        self.matrix = np.hstack(results)
      
    def _truncatedSVD(self, threshold: float = 0.90) -> np.ndarray:
        """Apply Truncated SVD to Explain 'n'% of total variance"""
        n_components = 2  # minimum components to begin
        ex_var = 0
        while ex_var < threshold:
            pc = TruncatedSVD(n_components=n_components)
            pc.fit_transform(self.matrix)
            ex_var = np.sum(pc.explained_variance_ratio_)
            n_components += 1
        logging.info(
            f"Total components {pc.n_components} with {ex_var:0.2f} variance explained"
        )
        self.matrix = pc.transform(self.matrix)

    def compute_similarity(self, metric: str = "cosine") -> np.ndarray:
        """Compute Similarity"""
        return pairwise_distances(self.matrix, metric=metric)

In [ ]:
def apply_similarity_calculation(name: str, features: List[str], metric: str) -> np.ndarray:
  """Compute User-Items Similarity Matrix
  Steps:
      - Construct User-Item Binary Vector for each input dataset
      - Apply truncatedSVD to determine 'n' components to explain m% of total variance
      - Compute cosine similarity
  """
  logging.info("=" * 50)
  logging.info(f"Computing USER-{name.upper()} Similarity Matrix...")
  logging.info(f"Input Features: {features}")
  SM = UserSimilarityMatrix(data[name])
  SM.get_user_item_matrix(data["max_users"], features)

  logging.info(f"Applying Truncated SVD: Input Shape: {SM.matrix.shape}...")
  SM._truncatedSVD()
  logging.info(f"Reduced User-Item Matrix Shape: {SM.matrix.shape}")

  # Compute pairwise user-similarity
  return SM.compute_similarity(metric=metric)

In [ ]:
user_interest = apply_similarity_calculation("interest", ["tag"], similarity_metric)

INFO:root:==================================================
INFO:root:Computing USER-INTEREST Similarity Matrix...
INFO:root:Input Features: ['tag']
INFO:root:Applying Truncated SVD: Input Shape: (10000, 747)...
INFO:root:Total components 164 with 0.90 variance explained
INFO:root:Reduced User-Item Matrix Shape: (10000, 164)


In [ ]:
user_assessment = apply_similarity_calculation("assessment", ["tag", "score"], similarity_metric)

INFO:root:==================================================
INFO:root:Computing USER-ASSESSMENT Similarity Matrix...
INFO:root:Input Features: ['tag', 'score']
INFO:root:Applying Truncated SVD: Input Shape: (10000, 55)...
INFO:root:Total components 20 with 0.91 variance explained
INFO:root:Reduced User-Item Matrix Shape: (10000, 20)


In [ ]:
%%time
user_courses = apply_similarity_calculation("course_tags", ["tag", "view"], similarity_metric)

INFO:root:==================================================
INFO:root:Computing USER-COURSE_TAGS Similarity Matrix...
INFO:root:Input Features: ['tag', 'view']
INFO:root:Applying Truncated SVD: Input Shape: (10000, 1000)...
INFO:root:Total components 264 with 0.90 variance explained
INFO:root:Reduced User-Item Matrix Shape: (10000, 264)


CPU times: user 12min 2s, sys: 5min 52s, total: 17min 55s
Wall time: 9min 43s


### Weighted averaging

In [ ]:
def compute_weighted_matrix(
    users: np.ndarray, assessments: np.ndarray, course: np.ndarray, weights: List[float]
) -> np.ndarray:
    """Compute Weighted Similarity Matrix where: weight_1 + weight_2 + weight_3 = 1"""
    return (
        (users * float(weights[0]))
        + (assessments * float(weights[1]))
        + (course * float(weights[2]))
    )

In [ ]:
def apply_weighted_similarity(i: np.ndarray, a: np.ndarray, c: np.ndarray, weights: List[float]) -> np.ndarray:
  """Compute Interest/Assessment/Courses Weighted Matrix"""
  logging.info("=" * 50)
  logging.info("Computing Weighted Similarity Matrix...")
  return compute_weighted_matrix(i, a, c, weights)

In [ ]:
weighted_matrix = apply_weighted_similarity(user_interest, user_assessment, user_courses, weights)

INFO:root:==================================================
INFO:root:Computing Weighted Similarity Matrix...


### Ranking

In [ ]:
def rank_similar_users(X: np.ndarray, top_n: int = 5) -> pd.DataFrame:
    """Apply Custom Pandas Function to Rank Top 'n' Users"""

    def custom_udf(X):
        """
        Custom Pandas function for using index/score to
        generate output results dataframe.
        """
        idx = np.argsort(X.values, axis=0)[::-1][1 : top_n + 1]
        return [
            str({"user": i, "score": X.astype(float).round(4).values[i]}) for i in idx
        ]

    # dimensions: users x top_n
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
    ranking = X.apply(custom_udf).T
    ranking.columns = [f"{i+1}" for i in ranking.columns]
    ranking["user_handle"] = ranking.index
    logging.info(f"User Ranking Dataframe Shape: {ranking.shape}")
    return ranking

In [ ]:
def apply_user_ranking(df: pd.DataFrame) -> pd.DataFrame:
  """Rank Users based on Similarity Metric"""
  logging.info("=" * 50)
  logging.info("Computing Weighted Similarity Matrix...")
  return rank_similar_users(df)

In [ ]:
%%time
rank_matrix = apply_user_ranking(weighted_matrix)

INFO:root:==================================================
INFO:root:Computing Weighted Similarity Matrix...
INFO:root:User Ranking Dataframe Shape: (10000, 6)


CPU times: user 17.2 s, sys: 28.8 ms, total: 17.3 s
Wall time: 17.2 s


### Save the similarity matrix into database

In [ ]:
def write_table(env: str, table: str, df: pd.DataFrame) -> None:
  """Write Table from SQLite Database"""
  conn = sqlite3.connect(database)
  cur = conn.cursor()
  df.to_sql(name=table, con=conn, if_exists="replace", index=False)

In [ ]:
def save(results: pd.DataFrame) -> None:
  """Write Output Data to Table in SQLite Database"""
  logging.info("=" * 50)
  logging.info("Updating similarity matrix in SQLite Database...")
  write_table(env, results_table, results)

In [ ]:
save(rank_matrix)

INFO:root:==================================================
INFO:root:Updating similarity matrix in SQLite Database...


## Adding Test sample

In [ ]:
def read_table(env: str, query: str) -> pd.DataFrame:
  """Query Table from SQLite Database"""
  conn = sqlite3.connect(database)
  cur = conn.cursor()
  cur.execute(query)
  df = pd.DataFrame(
      cur.fetchall(), columns=[column[0] for column in cur.description]
  )
  return df

In [ ]:
df_check_tags = read_table('dev', f"select * from course_tags")
df_check_tags.head()

,course_id,course_tags
0,12-principles-animation-toon-boom-harmony-1475,2d-animation
1,2d-racing-game-series-unity-5-1312,game-design
2,2d-racing-games-unity-volume-2-1286,game-art
3,2d-racing-games-unity-volume-2-1286,digital-painting
4,2d-racing-games-unity-volume-2-1286,image-editing


In [ ]:
# Sample Data
df_test = pd.DataFrame({'user_handle':['110','110','111','111'],
                        'user_match': ['112','113','157','145'],
                        'similarity': ['80.2','20.8','52.0','48.0']})

In [ ]:
# Write Similarty Results to Table
write_table('dev','test_table',df_test)

In [ ]:
# Read from Table
users = '110'
read_table('dev', f"select * from test_table where user_handle = {users}")

,user_handle,user_match,similarity
0,110,112,80.2
1,110,113,20.8


In [ ]:
# Add Index on Results Table (user_ranking)
conn = sqlite3.connect(database)
sql_table = f"""CREATE UNIQUE INDEX user_handle_index ON {results_table} (user_handle)"""
cur = conn.cursor()
cur.execute(sql_table)

## Manual evaluation

In [ ]:
# User content
user_assesments = pd.read_csv('user_assessment_scores.csv')
user_interest = pd.read_csv('user_interests.csv')
user_course_views = pd.read_csv('user_course_views.csv')
course_tags = pd.read_csv('course_tags.csv')

In [ ]:
input_user  = 9
read_table('dev', f"select * from {results_table} where user_handle = {input_user}")

,1,2,3,4,5,user_handle
0,"{'user': 9776, 'score': 1.0047}","{'user': 1171, 'score': 1.0034}","{'user': 225, 'score': 1.003}","{'user': 2211, 'score': 1.0027}","{'user': 3712, 'score': 1.0026}",9


In [ ]:
user_interest[user_interest['user_handle'] == input_user]

,user_handle,interest_tag,date_followed
297,9,cloud-computing,2017-11-06 16:55:35
298,9,devops,2017-11-06 16:55:35
299,9,microsoft-azure,2017-11-06 16:55:35
300,9,windows-azure,2017-11-06 16:55:35
301,9,azure-deployment,2017-11-06 16:55:35
302,9,azure,2017-11-06 16:55:35
303,9,cloud-computing,2017-11-06 16:55:35


In [ ]:
user_interest[user_interest['user_handle'] == 9776].head(10)

,user_handle,interest_tag,date_followed
292147,9776,c#,2017-06-21 07:36:34
292148,9776,data-analysis,2017-06-21 07:36:34
292149,9776,python,2017-06-21 07:36:34
292150,9776,vmware-vcloud,2017-06-21 07:36:34
292151,9776,security-auditing,2017-06-21 07:36:34
292152,9776,troubleshooting,2017-06-21 07:36:34
292153,9776,sql-server,2017-06-21 07:36:34
292154,9776,vmware-vsphere,2017-06-21 07:36:34
292155,9776,vsphere,2017-06-21 07:36:34
292156,9776,devops,2017-06-21 07:36:34


In [ ]:
user_assesments[user_assesments['user_handle'] == input_user]

,user_handle,assessment_tag,user_assessment_date,user_assessment_score
3752,9,azure-infrastructure-as-a-service,2017-11-06 17:06:03,141


In [ ]:
user_interest[user_interest['user_handle'] == 9776].head(10)

,user_handle,interest_tag,date_followed
292147,9776,c#,2017-06-21 07:36:34
292148,9776,data-analysis,2017-06-21 07:36:34
292149,9776,python,2017-06-21 07:36:34
292150,9776,vmware-vcloud,2017-06-21 07:36:34
292151,9776,security-auditing,2017-06-21 07:36:34
292152,9776,troubleshooting,2017-06-21 07:36:34
292153,9776,sql-server,2017-06-21 07:36:34
292154,9776,vmware-vsphere,2017-06-21 07:36:34
292155,9776,vsphere,2017-06-21 07:36:34
292156,9776,devops,2017-06-21 07:36:34


In [ ]:
user_course_views[user_course_views['user_handle'] == input_user]

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
233,9,2017-10-02,azure-adding-search-abilities-apps,140,Intermediate,2548
234,9,2017-10-03,azure-adding-search-abilities-apps,140,Intermediate,160
235,9,2017-10-05,terraform-getting-started,278,Beginner,998
236,9,2017-10-11,elasticsearch-analyzing-data,381,Intermediate,110
237,9,2017-10-11,elasticsearch-for-dotnet-developers,377,Beginner,3214
238,9,2017-10-16,terraform-getting-started,278,Beginner,678
239,9,2017-10-19,enterprise-search-using-apache-solr,936,Intermediate,2840
240,9,2017-10-20,enterprise-search-using-apache-solr,936,Intermediate,574
241,9,2017-10-27,intro-desktop-virtualization,221,Intermediate,2461
242,9,2017-11-04,getting-started-kubernetes,681,Beginner,2892


## Deploy and Inference

### Build the API

In [ ]:
%%writefile app.py

import os
import json
import sqlite3
import pandas as pd
from flask import Flask, request, jsonify

DATABASE_ENV = "dev"
DATABASE_NAME = "recommender_dev.db"
TABLE = "rank_matrix"

app = Flask(__name__)


def read_table(env: str, query: str) -> pd.DataFrame:
  """Query Table from SQLite Database"""
  conn = sqlite3.connect(DATABASE_NAME)
  cur = conn.cursor()
  cur.execute(query)
  df = pd.DataFrame(
      cur.fetchall(), columns=[column[0] for column in cur.description]
  )
  return df


class SimilarUsers:
    def __init__(self, user):
        self.user = user

    def fetch_user_from_db(self):
        """Fetch User Record from SQLite Database"""
        query = f"select * from {TABLE} where user_handle = {self.user}"
        print("Table", TABLE)
        return read_table(DATABASE_ENV, query)

    def get_payload(self):
        """Return JSON Payload containing Input User and Top
        Similar Users with associated similarity scores"""
        data = self.fetch_user_from_db()
        if data.shape[0] == 0:
            return {self.user_id: "No records found!"}
        else:
            return {str(self.user): list(data.loc[0].values.flatten()[:-1])}


@app.route("/api/similarity/", methods=["POST", "GET"])
def get_user_similarity():
    user = json.loads(request.get_data())["user_handle"]
    SU = SimilarUsers(user)
    results = SU.get_payload()
    return results


if __name__ == '__main__':
  app.run(debug=True)

Overwriting app.py


### Run the server API

In [ ]:
!chmod +x app.py
!nohup python3 app.py > output.log &

nohup: redirecting stderr to stdout


In [ ]:
!cat output.log

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 504-461-721


### Make post request

In [ ]:
def similarity(user_id: str, host: str = "0.0.0.0", port: int = 5000) -> json:

    """API call to flask app running on localhost
    and fetch top similar customers to the input customer(s)
    """
    url = f"http://{host}:{port}/api/similarity/"
    to_json = json.dumps({"user_handle": user_id})
    headers = {"content-type": "application/json", "Accept-Charset": "UTF-8"}
    response = requests.post(url, data=to_json, headers=headers)
    print(response.text)

In [ ]:
similarity(user_id='110')

{
  "110": [
    "{'user': 7785, 'score': 1.0065}", 
    "{'user': 5318, 'score': 1.0064}", 
    "{'user': 8165, 'score': 1.0055}", 
    "{'user': 6607, 'score': 1.0049}", 
    "{'user': 6586, 'score': 1.0046}"
  ]
}



### Make post request using CURL from command line

In [ ]:
!curl -X GET -H "Content-type: application/json" -d "{\"user_handle\":\"110\"}" "http://0.0.0.0:5000/api/similarity/"

{
  "110": [
    "{'user': 7785, 'score': 1.0065}", 
    "{'user': 5318, 'score': 1.0064}", 
    "{'user': 8165, 'score': 1.0055}", 
    "{'user': 6607, 'score': 1.0049}", 
    "{'user': 6586, 'score': 1.0046}"
  ]
}
